Where the actual experiment happens!

In [67]:
#do necessary imports
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
import torch
import json
import torch.nn as nn
import torch.optim as optim
from rean.utils import make_run_dir, to_serializable
from rean.data.Dataset import make_datasets
from rean.models.CNN import PlainCNN
from rean.models.P4 import P4CNN
from rean.models.RelaxedP4 import RelaxedP4CNN
from rean.train import train_full, evaluate
from rean.plot import LossPlot
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
#some params that should be the same across all experimental runs

group_order = 4
hidden_dim = 8
out_channels = hidden_dim
classes = 10
kernel_size = 3
num_gconvs = 4
num_epochs = 2
batch_size = 64
learning_rate = 0.001
gamma = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Experiment Running Loop #

In [70]:
#test full loop
models = ['PlainCNN', 'P4CNN', 'RelaxedP4CNN']
noise_types = ['none', 'iso', 'aniso']
stds = [0.1, 0.2]

for model_name in models:
    for noise_type in noise_types:
        for std, i in enumerate(stds):
            if noise_type == 'none':
                if i != 0: #only run noiseless once
                    continue
                train_noise = None
                noise_params = {"mean":0, "std":0, 'gamma':0}

            else:
                train_noise = noise_type
                test_noise = noise_type #could be none?
                noise_params = {'mean': 0, 'std': std, 'gamma': gamma}
            #make datasets
            train_ds, val_ds, test_ds, in_channels = make_datasets(train_noise=train_noise,
                                                            noise_params=noise_params,
                                                            group_order=group_order)
            #train model
            run_data, best_model = train_full(model_name=model_name,
                          train_ds=train_ds,
                          val_ds=val_ds,
                          in_channels=in_channels,
                          hidden_dim=hidden_dim,
                          out_channels=out_channels,
                          classes=classes,
                          num_gconvs=num_gconvs,
                          kernel_size=kernel_size,
                          group_order=group_order,
                          num_epochs=num_epochs,
                          batch_size=batch_size,
                          learning_rate=learning_rate,
                          device=device)
            run_data['noise_type'] = noise_type
            run_data['std'] = std

            run_dir = make_run_dir(model_name, noise_type, noise_params = noise_params, learning_rate = learning_rate)
            with open(run_dir / "run_data.json", "w") as f:
                json.dump(run_data, f, default = to_serializable, indent=2)

            #test the model
            test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, pin_memory =True)
            criterion = nn.CrossEntropyLoss()
            test_loss, test_acc = evaluate(best_model,  device, test_loader, criterion = criterion)
            test = {
                "test_loss": test_loss,
                "test_acc": test_acc
            }
            print(f"Test Loss: {test_loss}, Test Acc: {test_acc}")
            with open(run_dir / "test_data.json", "w") as f:
                json.dump(test, f, default = to_serializable, indent=2)



Epoch 1/2, Train Loss: 2.0325, Val Loss: 1.7915, Val Acc: 37.26%
Epoch 2/2, Train Loss: 1.6802, Val Loss: 1.5713, Val Acc: 42.47%
Test Loss: 1.5467161436080932, Test Acc: 42.7
Epoch 1/2, Train Loss: 2.1920, Val Loss: 2.1602, Val Acc: 19.78%
Epoch 2/2, Train Loss: 2.1554, Val Loss: 2.1340, Val Acc: 21.58%
Test Loss: 2.072044228363037, Test Acc: 24.03
Epoch 1/2, Train Loss: 2.0158, Val Loss: 1.8780, Val Acc: 28.91%
Epoch 2/2, Train Loss: 1.7791, Val Loss: 1.6324, Val Acc: 42.20%
Test Loss: 1.6109718307495118, Test Acc: 42.53
Epoch 1/2, Train Loss: 2.2537, Val Loss: 2.1532, Val Acc: 20.77%
Epoch 2/2, Train Loss: 2.1444, Val Loss: 2.1607, Val Acc: 19.88%
Test Loss: 3.0589671253204345, Test Acc: 13.14
2966 trainable parameters in P4CNN model
Epoch 1/2, Train Loss: 2.0218, Val Loss: 1.6514, Val Acc: 35.23%
Epoch 2/2, Train Loss: 1.4435, Val Loss: 1.3300, Val Acc: 49.27%
Test Loss: 1.306173657989502, Test Acc: 50.2
2966 trainable parameters in P4CNN model
Epoch 1/2, Train Loss: 2.2749, Val Lo